<b><font size = 6>  NIST Alternatives for Resilient Communities (ARC)
<table ><tr >
<td> <p style="font-size:16px">An interactive tool for developing alternative sets of actions that meet community resilience and cost goals, given hazard and interdependency information, and socio-economic data.</p></td>
<td> <img src="ARCOverviewGraphic.jpg"/> </td></tr></table>

------------------
# Introduction

This tool was produced within the NIST Community Resilience Program.  It is one of a range of NIST research products that includes guidance documents and other NIST-funded tools available to support community resilience planning:
* Community Resilience Planning Guide, providing a six-step process for developing community resilience plans (https://www.nist.gov/topics/community-resilience/planning-guide)
* EDGe\$, a tool for economic analysis (https://edges.nist.gov/)
* Center for Risk-based Community Resilience Planning (http://resilience.colostate.edu/in_core.shtml)

--------
# Setup--Python 

In [1]:
from lib.PythonSetup import *

import lib.myGlobals
import lib.myAMPL
import lib.mySolver
import lib.myDirectory
import lib.myVisualization
import lib.myWidgets
import lib.myStats

warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=RuntimeWarning)

--------
# Mathematical Programming Modeling Environment -- AMPL

## Setup--AMPL

In [2]:
config=ReadConfigFile()
ampl = lib.mySolver.environment(config['File paths']['AMPL'])

...Read NIST ARC configuration file
option version 'AMPL Version 20221013 (MSVC 19.29.30146.0, 64-bit)\
Licensed to Natl Institute of Standards and Tech, Ken Harrison <kenneth.harrison@nist.gov> (Hibiscus).\
Maintenance expires with version 20230331.\
Using license file "C:\Users\znf3\Downloads\ampl\ampl.lic".\
';
None


In [3]:
# This function provides the means to enter custom objectives and constraints later in the notebook
#@needs_local_scope
@register_cell_magic
def AMPL_CODE_BLOCK(line,cell):
    ampl.eval(cell)

In [4]:
ampl.setOption('solver', str(config['File paths']['Solver']))

In [5]:
myAMPL = lib.myAMPL.Object(ampl)

## Math Program -- Base Model

In [6]:
# Read in the base ARC model
print(str(Path('lib','ARC.mod')))
ampl.read(str(Path('lib','ARC.mod')))

lib\ARC.mod


In [7]:
basesnapshot = myAMPL.takeModelEntitySnapshot()

## Custom Objective Functions

In [8]:
%%AMPL_CODE_BLOCK

minimize RecoveryDisutilityObjectiveFxn:
	RecoveryDisutility;

minimize EV_TotalCostObjectiveFxn:
	EV_TotalCost;
    
minimize MGAObjectiveFxn:
	MGA;    

## Custom Constraints

In [9]:
%%AMPL_CODE_BLOCK

param EV_TotalCost_OVERBRACE 'Total expected costs upper limit [$]' default Infinity; 
CLimit:
 	   EV_TotalCost <= EV_TotalCost_OVERBRACE; 
        
param RecoveryDisutility_OVERBRACE 'recovery time upper limit [equivalent days]' default Infinity; 
TLimit:
 	   RecoveryDisutility <= RecoveryDisutility_OVERBRACE; 
              
param MCLIMIT_lb 'mitigation cost upper limit [$]' {I} default 0;
param MCLIMIT_ub 'mitigation cost lower limit [$]' {I} default Infinity;
MitConstraint {i in I}: MCLIMIT_lb[i] <= MitigationCostByNode[i] <= MCLIMIT_ub[i];

param RLIMIT_lb 'Increase in resistance lower limit [ft]' {I} default 0;
param RLIMIT_ub 'Increase in resistance upper limit [ft]' {i in I} default R_OVERBRACE[i];
ResistanceConstraint {i in I}: RLIMIT_lb[i] <= r[i] <= RLIMIT_ub[i];

--------
# Graphical User Interface (GUI)

## Setup--GUI

In [10]:
mainWidgets = lib.myWidgets.mainWidgets(ampl, myAMPL, basesnapshot)

In [11]:
mainWidgets.Events()

## Understand the Situation

In this section, the input files will be uploaded and the dependency between components will be visualized. 
The first tab is the <b>"Input Dataset"</b> tab with the options to "Select", "Preview" and "Upload" input files. Once the input files are selected, they can previewed and, if they look appropriate, can be uploaded by clicking on the "Upload" button. 
There is also an option to clear the selected files which can be done by clicking on the "Clear Selected Files" button. 

The next tab is the <b>"System Components"</b> tab where the parameters and sets will be displayed by clicking on the "Display" button. 

The last tab in this section is the <b>"System Dependency"</b> tab where the components and their interdependencies are shown as a networked graph. Nodes in the network represent the components of the system and are connected using directed links. Links show the interdependency between components. 
In order to visualize the network, first the layout of the graph should be selected from the provided dropdown widgets. For the "manual" layout, the "coordinates" file is also needed to be uploaded. If the images are available for the components, the "Show Images" check box should be marked and then the images should be uploaded. 

In [12]:
mainWidgets.Understand_the_Situation

## Run the Solver and Interactive Search

Here we begin by defining custom objective functions and constraints which can also be expanded by the user. Later, in the "Run Solver" tab, the list of all the defined objective functions and constraints will be available.

This section, consists of the three follwoing tabs:

* "Define Recovery"
* "Run Solver"
* "Results"

In the <b>"Define Recovery"</b> tab, the recovery end point should be selected by the user from the list of all the components in the system. 

The next tab is the <b>"Run Solver"</b> tab. There are two options: <b>"Benchmark Solution"</b> and <b>"Custom Solution"</b>. The benchmark solutions include some predefined scenarios such as "Do Nothing" or "Unlimited Budget" scenarion. For "Custom Solution", the user has the option of selecting the objective function along with the appropriate constraint. At this step, the user can either run the solver by clicking on the "Run Solver" button or by first adding an additional constraint by marking the <b>"Add additional constraint"</b> check box. 
In order to add additinal con
    straint, first the layout of the graph should be selected. Once the graph is displayed, the user can either <b>"Fix"</b> a variable or determine a <b>"Bound"</b> for a variable by clicking on a node on the graph. Also, the current constraints for each component can be removed by selecting the <b>"No restriction"</b> option. Once all additional constraints are defined, the model can be solved by clicking on the "Run Solver" button.
    
The list of all generated solutions are displayed at the end with the options of <b>"Rename"</b>, <b>"Delete"</b> or <b>"Save"</b> in the saved solutions directory. The saved solution directory should be specified by the user.
    
The last tab in this section is the <b>"Results"</b> tab in which the selected solutions in the list will be compared side by side. There are different options for comparison: They can be either compared in terms of <b>"Top level Info"</b> or <b>"Detailed Info"</b>. To compare the detailed information, there are two options of bar chart or network visualization for both mitigation and recovery stages. 

In [13]:
mainWidgets.Interactive_Solver

## Tools

The trade-off curve will be displayed in <b>"Tools"</b> section by click on the display button.

In [14]:
mainWidgets.Tools